In [24]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import math

"""
Node.csv: 노드 정보
Edge.csv: 엣지 정보

SampledNode.csv: 가중치 계산 시 사용되는 노드 정보
DataAnalysis.xlsx: 노드 세부 정보
"""

'\nNode.csv: 노드 정보\nEdge.csv: 엣지 정보\n\nSampledNode.csv: 가중치 계산 시 사용되는 노드 정보\nDataAnalysis.xlsx: 노드 세부 정보\n'

In [25]:
data = pd.read_json("./cve_list.txt")
data

,0,1
0,CVE-2017-1000503,"{'entities': [['V2_CVSS', 6.8], ['VULNERABILIT..."
1,CVE-2017-11418,"{'entities': [['DRIVER', 'dapur/apps/app_artic..."
2,CVE-2017-10887,"{'entities': [['CWE', 'CWE-426'], ['VULNERABIL..."
3,CVE-2017-9434,"{'entities': [['SOFTWARE', 'cryptopp'], ['SOUR..."
4,CVE-2017-17575,"{'entities': [['SOURCECODE', 'vendor_details.p..."
...,...,...
448,CVE-2017-16014,"{'entities': [['VULNERABILITY', 'dos'], ['V2_C..."
449,CVE-2017-11439,"{'entities': [['VENDER', 'sitecore'], ['CWE', ..."
450,CVE-2017-2789,"{'entities': [['SOFTWARE', 'ichitaro office'],..."
451,CVE-2017-3156,"{'entities': [['VENDER', 'apache'], ['CWE', 'C..."


In [26]:
node = pd.DataFrame({"Label": data[0]})
node

,Label
0,CVE-2017-1000503
1,CVE-2017-11418
2,CVE-2017-10887
3,CVE-2017-9434
4,CVE-2017-17575
...,...
448,CVE-2017-16014
449,CVE-2017-11439
450,CVE-2017-2789
451,CVE-2017-3156


In [27]:
entities = []
for d in data[1]:
    d['entities'] = dict(d['entities'])
    entities.append(d['entities'])
entities = pd.DataFrame(entities)
entities

,V2_CVSS,VULNERABILITY,STRUCT,CWE,IMPACT,VENDER,DRIVER,SOURCECODE,SOFTWARE,FUNCTION
0,6.8,execcode,race condition,CWE-362,6.4,jenkins,NaN,NaN,NaN,NaN
1,7.5,sql,NaN,CWE-89,6.4,NaN,dapur/apps/app_article/controller/,article_list.php,fiyo cms,NaN
2,9.3,priv,untrusted search path,CWE-426,10.0,NaN,NaN,NaN,book walker,NaN
3,5.0,unknown,NaN,CWE-125,2.9,NaN,NaN,zinflate.c,crypto,NaN
4,7.5,sql,NaN,CWE-89,6.4,NaN,NaN,item_details.php,fs groupon clone,NaN
...,...,...,...,...,...,...,...,...,...,...
448,5.0,dos,NaN,CWE-388,2.9,NaN,NaN,NaN,http-proxy,NaN
449,3.5,xss,NaN,CWE-79,2.9,sitecore,shell/applications/tools/run,NaN,sitecore,NaN
450,7.5,execcode overflow,NaN,CWE-119,6.4,justsystems,NaN,NaN,ichitaro office,NaN
451,5.0,unknown,NaN,CWE-361,2.9,apache,NaN,NaN,cxf,NaN


In [28]:
NodeTable = pd.concat([node, entities], axis = 1)
NodeTable.to_csv("Node.csv", index = False)

In [29]:
# feature들의 NaN 값 frequency
NaN_COUNT = pd.DataFrame({"NaN" : entities.isnull().sum()})

# Label 칼럼 값들의 frequency
CVE_ID = pd.DataFrame(node.value_counts())

# feature들의 label을 index list에 저장
index = entities.keys()

# .xlsx 파일 생성
with pd.ExcelWriter('DataAnalysis.xlsx') as writer:
    NodeTable.to_excel(writer, sheet_name = 'NodeTable')
    NaN_COUNT.to_excel(writer, sheet_name = 'NaN_COUNT')
    CVE_ID.to_excel(writer, sheet_name = 'CVE_ID')
    for i in index:
        pd.DataFrame(entities[i].value_counts()).to_excel(writer, sheet_name = i)

In [30]:
Sample = NodeTable.loc[:, ['Label', 'V2_CVSS', 'VULNERABILITY', 'STRUCT', 'CWE', 'IMPACT', 'VENDER', 'SOFTWARE']]
Sample

,Label,V2_CVSS,VULNERABILITY,STRUCT,CWE,IMPACT,VENDER,SOFTWARE
0,CVE-2017-1000503,6.8,execcode,race condition,CWE-362,6.4,jenkins,NaN
1,CVE-2017-11418,7.5,sql,NaN,CWE-89,6.4,NaN,fiyo cms
2,CVE-2017-10887,9.3,priv,untrusted search path,CWE-426,10.0,NaN,book walker
3,CVE-2017-9434,5.0,unknown,NaN,CWE-125,2.9,NaN,crypto
4,CVE-2017-17575,7.5,sql,NaN,CWE-89,6.4,NaN,fs groupon clone
...,...,...,...,...,...,...,...,...
448,CVE-2017-16014,5.0,dos,NaN,CWE-388,2.9,NaN,http-proxy
449,CVE-2017-11439,3.5,xss,NaN,CWE-79,2.9,sitecore,sitecore
450,CVE-2017-2789,7.5,execcode overflow,NaN,CWE-119,6.4,justsystems,ichitaro office
451,CVE-2017-3156,5.0,unknown,NaN,CWE-361,2.9,apache,cxf


In [31]:
def ProcessCVSS(x):
    if 0 <= x and x < 2.5:
        return 0
    elif 2.5 <= x and x < 5:
        return 2.5
    elif 5 <= x and x < 7.5:
        return 5
    else:
        return 7.5
    
for i in range(0, len(Sample)):
    Sample.loc[i, 'V2_CVSS'] = ProcessCVSS(Sample.loc[i, 'V2_CVSS'])
    
Sample
Sample.to_csv("SampledNode.csv", index = False)

In [32]:
software = []
idx = 0

for i in Sample.loc[:, 'SOFTWARE']:
    if type(i) == type('str'):
        l = i.split(" ")
        for j in l:
            software.append([])
            software[idx].append(j)
    else:
        software.append([])
        software[idx].append(math.nan)
    idx += 1

software = software[: idx]
software

[[nan],
 ['fiyo', 'cms'],
 ['book', 'walker'],
 ['crypto'],
 ['fs', 'groupon', 'clone'],
 ['relm'],
 ['android'],
 ['binutils'],
 ['zfs', 'storage', 'appliance', 'kit'],
 ['libav'],
 ['security', 'access', 'manager', 'appliance'],
 ['application_helper.rb', ''],
 ['cubecart'],
 ['ios'],
 ['android'],
 ['backintime'],
 ['file', 'compact'],
 ['opentext', 'documentum', 'content', 'server'],
 ['flatcore'],
 ['arris', 'nvg599', 'device'],
 ['synology', 'chat'],
 ['windows', '8.1'],
 ['asyncos', 'software'],
 ['nylas', 'mail', 'lives'],
 ['timidity'],
 ['httpd'],
 ['ios'],
 ['odl-l2switch-switch'],
 ['fs', 'lynda', 'clone'],
 ['elastic', 'services', 'controllers'],
 ['quantastor', 'v4', 'virtual', 'appliance'],
 ['sharepoint', 'server'],
 [nan],
 ['ar100'],
 ['swfcombine'],
 ['yara'],
 ['prime', 'infrastructure'],
 ['dotnetnuke'],
 ['edr-810'],
 ['afm'],
 ['uclibc'],
 ['gnome-session'],
 ['wireshark'],
 ['android'],
 ['securimage'],
 ['jbig2dec'],
 ['mlalchemy'],
 ['windows', '8.1'],
 ['prox

In [33]:
def JaccardSimilarity(list1, list2):
    NaN = [math.nan]
    if list1 == NaN or list2 == NaN:
        return 0
    else:
        union = set(list1).union(set(list2))
        intersection = set(list1).intersection(set(list2))
        return len(intersection)/len(union)

In [34]:
def Similarity(list1, list2):
    count = 0
    for i in range(1,7):
        if type(list1[i]) == type(list2[i]):
            if type(list1[i]) == type('str'):
                if list1[i] == list2[i]:
                    count += 1
            else:
                if not(math.isnan(list1[i]) or math.isnan(list2[i])):
                    if list1[i] == list2[i]:
                        count += 1
    return count

In [35]:
weight = []
idx2 = 0

for i in range(len(Sample)):
    for j in range(i+1, len(Sample)):
        s = Similarity(Sample.loc[i, :].tolist(), Sample.loc[j, :].tolist())
        s += JaccardSimilarity(software[i], software[j])
        s /= 7
        s = round(s, 1)
        if(s != 0):
            weight.append([])
            weight[idx2].append(i)
            weight[idx2].append(j)
            weight[idx2].append(s)
            idx2 += 1
            
weight = pd.DataFrame(weight)
weight.columns = ['Source', 'Target', 'Weight']
weight

,Source,Target,Weight
0,0,1,0.1
1,0,3,0.1
2,0,4,0.1
3,0,7,0.3
4,0,8,0.1
...,...,...,...
57490,448,451,0.3
57491,448,452,0.1
57492,449,451,0.1
57493,449,452,0.3


In [36]:
weight.to_csv("Edge.csv", index = False)